In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import os 
from sklearn.metrics import confusion_matrix, auc
from sklearn import metrics
from xgboost import plot_importance
from sklearn.tree import DecisionTreeClassifier
import swifter


/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
!ls

input		sub_xgb_hist_pos_weight.csv.gz	xgboost.ipynb
lightgbm.ipynb	tree.ipynb			xgboost_outout.ipynb


In [3]:
PATH = "input"

In [4]:
!ls {PATH}

sample_submission.csv	   test_supplement.csv	    train_sample.csv
sample_submission.csv.zip  test_supplement.csv.zip  train_sample.csv.zip
test.csv		   train.csv
test.csv.zip		   train.csv.zip


In [5]:
def dataPreProcessTime(df):
    df['click_time'] = pd.to_datetime(df['click_time']).dt.date
    df['click_time'] = df['click_time'].swifter.apply(lambda x: x.strftime('%d%H%M')).astype(int)
    
    return df

In [6]:
start_time = time.time()

print("reading data")
train = pd.read_csv(PATH+"/train.csv", skiprows=147403891, nrows=37500000) 
# nrows in "train.csv" is: 184903891
train.columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time', 'is_attributed']
test = pd.read_csv(PATH+"/test.csv")



reading data


/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
print('[{}] Finished to load data'.format(time.time() - start_time))

train = dataPreProcessTime(train)
test = dataPreProcessTime(test)

[69.2905056476593] Finished to load data


Pandas Apply: 100%|██████████| 18790469/18790469 [01:12<00:00, 259163.78it/s]


In [8]:
!ls

input		sub_xgb_hist_pos_weight.csv.gz	xgboost.ipynb
lightgbm.ipynb	tree.ipynb			xgboost_outout.ipynb


In [9]:
y = train['is_attributed']
train.drop(['ip', 'is_attributed', 'attributed_time'], axis=1, inplace=True)

sub = pd.DataFrame()
sub['click_id'] = test['click_id']
test.drop(['ip', 'click_id'], axis=1, inplace=True)

In [10]:
print("training set | data types: ")
print(train.info())
print("train size: ", len(train))
print("test size: ", len(test))

print('[{}] Start Decision Tree Classifier Training'.format(time.time() - start_time))

training set | data types: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37499999 entries, 0 to 37499998
Data columns (total 5 columns):
app           int64
device        int64
os            int64
channel       int64
click_time    int64
dtypes: int64(5)
memory usage: 1.4 GB
None
train size:  37499999
test size:  18790469
[332.22058868408203] Start Decision Tree Classifier Training


In [11]:
x1, x2, y1, y2 = train_test_split(train, y, test_size=0.1, random_state=84)


In [12]:
x1.head()

,app,device,os,channel,click_time
3876180,3,1,6,280,90000
36413389,18,1,10,107,90000
18997479,9,1,36,127,90000
35793788,3,1,53,130,90000
926462,15,1,13,480,90000


In [13]:
y1.head()

3876180     0
36413389    0
18997479    0
35793788    0
926462      0
Name: is_attributed, dtype: int64

In [14]:
!ls

input		sub_xgb_hist_pos_weight.csv.gz	xgboost.ipynb
lightgbm.ipynb	tree.ipynb			xgboost_outout.ipynb


# Decison Tree: equal weight

In [15]:
clf = DecisionTreeClassifier(random_state=0)

In [16]:
clf.fit(x1,y1)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [17]:
y_pred = clf.predict(x2)

In [18]:
cm = confusion_matrix(y2, (y_pred>0.5))
cm

array([[3739954,     688],
       [   7128,    2230]])

In [19]:
baseline_predictions = np.zeros(len(y2))

In [20]:
fpr, tpr, thresholds = metrics.roc_curve(y2, baseline_predictions)
metrics.auc(fpr, tpr)


0.5

In [21]:
fpr, tpr, thresholds = metrics.roc_curve(y2, y_pred)
metrics.auc(fpr, tpr)

0.61905742807199649

# Decison Tree: class weight

In [15]:
clf = DecisionTreeClassifier(random_state=0, class_weight={0:1, 1: 15})

In [16]:
clf.fit(x1,y1)

DecisionTreeClassifier(class_weight={0: 1, 1: 15}, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [17]:
y_pred = clf.predict(x2)

In [18]:
cm = confusion_matrix(y2, (y_pred>0.5))
cm

array([[3719454,   21188],
       [   2782,    6576]])

In [19]:
fpr, tpr, thresholds = metrics.roc_curve(y2, y_pred)
metrics.auc(fpr, tpr)

0.84852499342829957

In [25]:
train.head()

,app,device,os,channel,click_time
0,3,1,18,130,90000
1,10,1,9,377,90000
2,18,1,17,121,90000
3,12,1,17,497,90000
4,3,1,13,280,90000


In [26]:
y.value_counts()

0    37405507
1       94492
Name: is_attributed, dtype: int64

In [29]:
94492/37405507*100

0.25261520984062585

In [27]:
37405507/94492

395.85898277102825

In [30]:
clf = DecisionTreeClassifier(random_state=0, class_weight={0:1, 1: 150})

clf.fit(x1,y1)

DecisionTreeClassifier(class_weight={0: 1, 1: 150}, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [31]:
y_pred = clf.predict(x2)

In [32]:
cm = confusion_matrix(y2, (y_pred>0.5))
cm

array([[3688716,   51926],
       [   1730,    7628]])

In [33]:
fpr, tpr, thresholds = metrics.roc_curve(y2, y_pred)
metrics.auc(fpr, tpr)

0.90062493200107396